# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [2]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']
print("ok")

ok


# Making random queries to the API

In [3]:
challenge_id = 'RunxingJia78911' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
print(queries)
query_results = query(challenge_id, queries)

print(query_results)


[[ 1  1  1 ... -1 -1  1]
 [ 1  1 -1 ...  1 -1 -1]
 [-1  1  1 ...  1  1  1]
 ...
 [ 1  1  1 ...  1 -1  1]
 [-1 -1  1 ... -1 -1 -1]
 [-1 -1 -1 ...  1 -1  1]]
[-11.  18.   9. -14.  -2.   5.  10.  16.  20.   6.  -5.  19.   4.  -1.
 -15.  -2. -35.  -7.   0.   1. -14.  -1.  17.  -7.  -5.  22.  14.   2.
 -17.  -8.   4. -27.  10. -12.  44. -14.  15. -46.  -5.   5.   7.  12.
  16.   6.  18.   7. -21.  -2.  -6.  -0. -18. -20.   4. -42.   4.   3.
  12.  -5.   6.   0.  13. -19.  -5. -44. -14.  25. -26. -12. -30.  13.
 -14. -40. -13. -11. -10.   7.  -2. -23.  19.   2. -19.  -2. -21.   4.
  18.  -2. -22.   1.  -3.   1.  23.  10.  16.  32.   8.   8. -10.   4.
  -2. -17.   5.   2. -10. -10.   3.  -1.  -5.  11.   7.   7. -38.   7.
  -2.  -7.  11.   3.   4.  11.  -2.   3.  -1.  12.  22. -19.  -7. -11.
   5.   9. -10.   9. -13.  -6.  -7.  -3.  10.   7.   4. -38.  31.  10.
   7. -20. -10.   6.  -2.   5.  -6. -49. -20. -20. -14.  16.  10.   6.
  13. -18.  37.   9.  28.   1.   8. -17.  15.   8.  -2.  21.  3

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [16]:
best_query_number = np.argmax(query_results)
best_query = queries[best_query_number]
best_query_result = query(challenge_id, best_query, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")


Reconstruction attack achieves fraction 0.46 correct values


In [48]:
from scipy.optimize import linprog
import numpy as np

# 假设 challenge_id 和 n 的定义
challenge_id = 'kivkiv677899kivki'
n = 100  # 数据集大小
num_queries = 2*n  # 查询次数上限

# 生成随机查询向量，每个查询向量长度为 n
queries = np.random.choice([-1, 1], size=(num_queries, n))
print("queries:",queries.shape)

# 获取查询结果
query_results = query(challenge_id, queries)
print("query_results:",query_results.shape)

# 构建线性规划问题
c = np.ones(n)  # 目标是最小化L1距离
epsilon = 30  # 设置容差，先尝试更大的值

# 使用简化的正向约束
A_ub = queries  # 只保留正向查询矩阵，简化问题
b_ub = query_results + epsilon  # 只使用正向约束的上界

print("A_ub:",A_ub.shape)
print("b_ub:",b_ub.shape)

# 设置变量边界，限制 x 在 [0, 1] 之间
bounds = [(-1, 1) for _ in range(n)]  # 设置为 [0, 1]，方便之后用 np.sign 转换为 -1 和 1

# 使用 linprog 求解线性规划
res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')  # 使用 highs 方法求解

# 提取重建结果
if res.success:
    print("success!!")
    x_reconstructed = np.sign(res.x)  # 将结果转换为 -1 或 1
else:
    print(f"Linear programming failed: {res.message}")
    x_reconstructed = np.random.choice([-1, 1], size=n)  # 如果失败，使用随机结果

# 输出重建结果
print(f"Reconstructed x: {x_reconstructed}")
best_query_result = query(challenge_id, x_reconstructed, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")


queries: (200, 100)
query_results: (200,)
A_ub: (200, 100)
b_ub: (200,)
Linear programming failed: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is Basic)
Reconstructed x: [ 1  1 -1  1 -1  1 -1  1  1  1  1 -1 -1  1 -1 -1 -1  1 -1  1  1 -1  1 -1
 -1  1  1 -1 -1 -1  1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1
 -1  1 -1 -1  1 -1  1 -1 -1 -1  1 -1 -1 -1 -1  1  1 -1  1 -1 -1 -1  1 -1
 -1 -1  1 -1  1  1 -1 -1  1 -1  1  1 -1  1 -1 -1 -1  1  1  1  1 -1 -1 -1
 -1  1 -1 -1]

Reconstruction attack achieves fraction 0.51 correct values


In [6]:
import numpy as np
from scipy.optimize import linprog

n = 100        
t = 2 * n       
lambda_ = 10    
challenge_id = 'RunxingJia78911'


queries = np.random.choice([-1, 1], size=(t, n))
print("queries:",queries.shape)


query_results = query(challenge_id, queries)
print("query_results:",query_results.shape)

num_vars = n + t 

#Coefficients of the objective function
c = np.hstack([np.zeros(n), np.ones(t)])

print("c:",c.shape)
A_ub = []
b_ub = []

for j in range(t):
    q = queries[j]
    y = query_results[j]
    
    # q_j, x⟩ - y_j - e_j ≤ 0
    A_row = np.hstack([q, np.zeros(t)])
    A_row[n + j] = -1  #  -e_j
    A_ub.append(A_row)
    b_ub.append(y)
    
    # -⟨q_j, x⟩ + y_j - e_j ≤ 0
    A_row = np.hstack([-q, np.zeros(t)])
    A_row[n + j] = -1  #  -e_j
    A_ub.append(A_row)
    b_ub.append(-y)

A_ub = np.array(A_ub)
b_ub = np.array(b_ub)

print("A_ub:",A_ub.shape)
print("b_ub:",b_ub.shape)

bounds = [(-1, 1)] * n + [(0, None)] * t

result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
print(result.x.shape)

if result.success:
    x_estimated = result.x[:n]
    #  -1 or 1
    x_reconstructed = np.sign(x_estimated)
    
else:
    print("fail!!!")


print(f"Reconstructed x: {x_reconstructed}")
best_query_result = query(challenge_id, x_reconstructed, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")



queries: (200, 100)
query_results: (200,)
c: (300,)
A_ub: (400, 300)
b_ub: (400,)
(300,)
Reconstructed x: [ 1.  1.  1. -1.  1. -1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1. -1.  1.
  1. -1.  1.  1. -1. -1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1. -1. -1.
  1.  1.  1. -1. -1.  1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1.
 -1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1.  1. -1.  1.  1.
  1.  1.  1.  1. -1.  1.  1.  1. -1. -1.]

Reconstruction attack achieves fraction 0.89 correct values
